In [27]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 109.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 26.5 MB/s eta 0:00:00


In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [3]:
import mlflow
import mlflow.sklearn


In [4]:
# !rm -r mlflow_logs

In [4]:
!git clone 'https://github.com/Saksham0803/Week5'

Cloning into 'Week5'...


In [5]:
mlflow.set_tracking_uri("file:./mlflow_logs/mlruns")  # this sets a local directory for logs


In [6]:
%cd Week5

/home/jupyter/Week5


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
data = pd.read_csv('Data/iris.csv')

In [9]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [11]:
X = data.drop(columns=["species"])
y = data["species"]
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=8)

clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

print("Accuracy:", accuracy_score(y_test, clf.predict(X_test)))

Accuracy: 0.8947368421052632


In [12]:
!pwd

/home/jupyter/Week5


HyperParameter Tuning

In [13]:
with mlflow.start_run():
    X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42)
    
    clf_hpt = DecisionTreeClassifier()
    
    # Define the hyperparameter grid
    param_grid = {
        "criterion": ["gini", "entropy", "log_loss"],
        "max_depth": [None, 3, 5, 10],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4],
        "max_features": [None, "sqrt", "log2"]
    }
    
    # Set up GridSearchCV
    grid_search = GridSearchCV(estimator=clf_hpt,
                               param_grid=param_grid,
                               cv=5,
                               scoring='accuracy',
                               n_jobs=-1,
                               verbose=1)
    
    # Fit to the training data
    grid_search.fit(X_train, y_train)
    
    # Best estimator and score
    print("Best Parameters:", grid_search.best_params_)
    print("Best Cross-Validated Score:", grid_search.best_score_)
    
    # Evaluate on test set
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    print("Test Accuracy:", accuracy_score(y_test, y_pred))
    model_acc = accuracy_score(y_test, y_pred)
    
    
    for param_name, param_value in grid_search.best_params_.items():
        mlflow.log_param(param_name, param_value)

    mlflow.log_metric('accuracy', model_acc)
    input_example = pd.DataFrame(X_test.iloc[:2]) 
    mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path='best_model',
        input_example=input_example,
        signature=mlflow.models.signature.infer_signature(X_test, y_pred)
    )


Fitting 5 folds for each of 324 candidates, totalling 1620 fits


2025/07/27 16:08:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Best Parameters: {'criterion': 'gini', 'max_depth': 5, 'max_features': 'log2', 'min_samples_leaf': 4, 'min_samples_split': 2}
Best Cross-Validated Score: 0.9549407114624506
Test Accuracy: 0.9736842105263158


In [14]:
import joblib

In [15]:
joblib.dump(best_model, "Model/model_wk5.joblib")

['Model/model_wk5.joblib']

In [25]:
!gsutil cp model_wk5.joblib gs://mlops-course-vast-advantage-461609-t2-unique/Wk5/Model

Copying file://model_wk5.joblib [Content-Type=application/octet-stream]...
/ [1 files][  2.7 KiB/  2.7 KiB]                                                
Operation completed over 1 objects/2.7 KiB.                                      


In [34]:
!pwd

/home/jupyter


In [35]:
ls


'Mlflow logs'/                           artifacts/   iris_pipeline/
 SDK_Custom_Container_Prediction.ipynb   data/        model_wk5.joblib
 Week5.ipynb                             iris.csv     week3.ipynb


In [36]:
cd Mlflow\ logs

/home/jupyter/Mlflow logs


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [37]:
ls

tmp/


In [38]:
cd tmp

/home/jupyter/Mlflow logs/tmp


In [39]:
ls

mlruns/


In [40]:
cd mlruns

/home/jupyter/Mlflow logs/tmp/mlruns


In [41]:
ls

0/


In [42]:
cd 0

/home/jupyter/Mlflow logs/tmp/mlruns/0


In [43]:
ls

49df6abf655142628359a9721e45ee5d/  b8edf2d470334051a724878e6fdc3989/  models/
a77da93ee23944efade49f5ab416c726/  meta.yaml


In [44]:
cd 49df6abf655142628359a9721e45ee5d

/home/jupyter/Mlflow logs/tmp/mlruns/0/49df6abf655142628359a9721e45ee5d


In [45]:
ls

artifacts/  meta.yaml  metrics/  outputs/  params/  tags/


In [48]:
mlflow.end_run()

In [16]:
!pwd

/home/jupyter/Week5


In [17]:
!git add .

# https://github.com/Saksham0803/Week5

In [18]:
!git commit -m 'week5'

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: empty ident name (for <jupyter@instance-20250601-170513.us-central1-a.c.vast-advantage-461609-t2.internal>) not allowed


In [19]:
!git config --global user.email "21f3002170@ds.study.iitm.ac.in"

In [ ]:
!git config --global user.name "Sak"